source: [LeetCode](https://leetcode.com/problems/word-search-ii/?envType=study-plan-v2&envId=top-interview-150)


# LeetCode 212 — Word Search II

**Difficulty:** Hard  
**Core Concepts:** Trie + DFS Backtracking + Pruning  
**Key Insight:** Search *many words at once* instead of one-by-one

---

## 🧠 Intuition

A naive solution would:
- For each word → run DFS on the board  
❌ This leads to **TLE** (`O(#words × board DFS)`)

Instead:
- Build a **Trie** of all words
- Run DFS **once per board cell**
- Traverse the Trie **while traversing the board**

This allows:
- Prefix pruning
- Early termination
- Shared computation across words

---

## 🏗️ Data Structure

### Trie Node
- `children`: dict(char → TrieNode)
- `word`: store full word at terminal node (or `is_end`)

```text
Words = ["oath", "pea", "eat", "rain"]

Trie:
root
 ├── o
 │    └── a
 │         └── t
 │              └── h (word="oath")
 └── e
      └── a
           └── t (word="eat")
```

---

## 🚀 Approach (High Level)

### Step 1: Build Trie

* Insert all words into Trie
* Store full word at terminal nodes (optimization)

### Step 2: DFS from each board cell

* Start DFS only if `board[i][j]` exists in Trie root
* Move in 4 directions (up, down, left, right)
* Mark cell as visited temporarily
* Move Trie pointer along with board traversal

### Step 3: Collect answers

* If Trie node contains a word → add to result
* Set `node.word = None` to avoid duplicates

### Step 4: Pruning

* If Trie node has no children → remove it (optional but fast)

---

## ✅ Python Implementation (Optimized Pattern)

```python
class TrieNode:
    def __init__(self):
        self.children = {}
        self.word = None

class Solution:
    def findWords(self, board, words):
        root = TrieNode()

        # Build Trie
        for w in words:
            curr = root
            for ch in w:
                if ch not in curr.children:
                    curr.children[ch] = TrieNode()
                curr = curr.children[ch]
            curr.word = w

        ROWS, COLS = len(board), len(board[0])
        res = []

        def dfs(r, c, node):
            ch = board[r][c]
            if ch not in node.children:
                return

            nxt = node.children[ch]

            if nxt.word:
                res.append(nxt.word)
                nxt.word = None  # avoid duplicates

            board[r][c] = "#"  # mark visited

            for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < ROWS and 0 <= nc < COLS and board[nr][nc] != "#":
                    dfs(nr, nc, nxt)

            board[r][c] = ch  # restore

            # Trie pruning
            if not nxt.children:
                node.children.pop(ch)

        for r in range(ROWS):
            for c in range(COLS):
                dfs(r, c, root)

        return res
```

---

## ⏱️ Time & Space Complexity

### Time

* **Best / Average:** `O(M × N × 4^L)` (heavily pruned)
* **Much faster than** `O(#words × board DFS)`

### Space

* Trie: `O(total characters in words)`
* DFS recursion: `O(L)`

Where:

* `M × N` = board size
* `L` = max word length

---


# My solution

In [ ]:
class TrieNode:
    def __init__(self):
        self.nexts = {}
        self.is_end = False

class Solution(object):
    def findWords(self, board, words):
        """
        :type board: List[List[str]]
        :type words: List[str]
        :rtype: List[str]
        """
        # Build the Trie
        self.root = TrieNode()
        for word in words:
            curr = self.root
            for l in word:
                if l not in curr.nexts:
                    curr.nexts[l] = TrieNode()
                curr = curr.nexts[l]
            curr.is_end = True
        # board 
        m = len(board)
        n = len(board[0])
        # DFS
        words = set()
        visited = set()
        def dfs(i, j, curr, curr_word):
            if i < 0 or j < 0 or i >= m or j >= n: return
            if board[i][j] not in curr.nexts: return
            visited.add((i, j))
            new_word = curr_word + board[i][j]
            next_curr = curr.nexts[board[i][j]]
            if next_curr.is_end: words.add(new_word)
            if (i+1, j) not in visited:
                dfs(i+1, j, next_curr, new_word)
            if (i-1, j) not in visited:
                dfs(i-1, j, next_curr, new_word)
            if (i, j+1) not in visited:
                dfs(i, j+1, next_curr, new_word)
            if (i, j-1) not in visited:
                dfs(i, j-1, next_curr, new_word)
            visited.discard((i,j))
        for i in range(m):
            for j in range(n):
                if board[i][j] in self.root.nexts:
                    dfs(i, j, self.root, "")
        return list(words)
            
        